In [ ]:
# !pip install -e ../.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from ExoHunter.params import *
from ExoHunter.cleaner import Cleaner
from ExoHunter.trainer import Trainer

/Users/lorcan/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def get_kaggle_data(data_name='kaggle', test_size=0.2, drive=0):
    train_path = os.path.join(DRIVE[drive], 'raw_data', data_name, FILEPATHS[data_name][0])
    test_path = os.path.join(DRIVE[drive], 'raw_data', data_name, FILEPATHS[data_name][1])
    train_data = pd.read_csv(train_path)
    data_test = pd.read_csv(test_path)
    data_train, data_val = train_test_split(train_data, test_size=test_size)
    return data_train, data_val, data_test

In [3]:
def min_window(data):
    temp = data.drop(columns='LABEL').T
    minim = temp[temp>-660].count().min()
    return minim

In [4]:
def get_nasa_data(data_name='nasa', test_size=0.2, drive=0):
    exo_path = os.path.join(DRIVE[drive], 'raw_data', data_name, FILEPATHS[data_name][0][0])
    non_exo_path = os.path.join(DRIVE[drive], 'raw_data', data_name, FILEPATHS[data_name][0][1])
    
    exo_data = pd.read_csv(exo_path)
    non_exo_data = pd.read_csv(non_exo_path)
    exo_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
    non_exo_data.drop(columns=['Unnamed: 0'], inplace=True)
    
    exo_data = exo_data.T
    non_exo_data = non_exo_data.T
    exo_data[['LABEL']] = 2
    non_exo_data[['LABEL']] = 1
    
    all_data = pd.concat([exo_data, non_exo_data])
    minim = min_window(all_data)
    print(minim)
    
    labels = all_data[['LABEL']]
    all_data = all_data.iloc[:, :minim]
    all_data[['LABEL']] = labels
    
    train_data, test_data = train_test_split(all_data, test_size=test_size)

    return train_data, test_data

In [5]:
def get_raw_data(data_name='nasa', test_size=0.2, drive=0):
    if data_name == 'kaggle':
        return get_kaggle_data(test_size=test_size, drive=drive)
    if data_name == 'nasa':
        return get_nasa_data(test_size=test_size, drive=drive)
    return None

In [6]:
def get_proc_data(data_name='nasa', test_size=0.2, drive=0):
    train_path = os.path.join(DRIVE[drive],'processed_data', data_name, FILEPATHS[data_name][1][0])
    test_path = os.path.join(DRIVE[drive],'processed_data', data_name, FILEPATHS[data_name][1][1])
    if data_name=='nasa':
        train_data = pd.read_csv(train_path, index_col='Index')
        data_test = pd.read_csv(test_path, index_col='Index')
    else:
        train_data = pd.read_csv(train_path)
        data_test = pd.read_csv(test_path)
    
    return train_data, data_test

In [7]:
def get_data(data_name='nasa', test_size=0.2, drive=0, raw=0):
    if raw:
        return get_raw_data(data_name, test_size, drive)
    return get_proc_data(data_name, test_size, drive)

In [8]:
def get_Xy(data):
        X = data.drop(columns='LABEL')
        y = data['LABEL'].map({1:0, 2:1})
        return X, y

In [9]:
cleaner = Cleaner()

In [ ]:
train_nasa, test_nasa = cleaner.get_data()

In [12]:
train_raw_nasa, test_raw_nasa = cleaner.get_data('nasa', 0.1, 0, 1)

In [10]:
train_raw_kaggle, test_raw_kaggle = cleaner.get_data('kaggle', 0.2, 0, 1)

In [ ]:
# FILEPATHS = {
#     'kaggle': ['exoTrain.csv', 'exoTest.csv'],
#     'nasa': [['nasa_exo_wo_label_df.csv', 'nasa_non_exo_wo_label_df.csv'], ['nasaTrain.csv', 'nasaTest.csv']]
# }
# train_nasa, test_nasa = get_data()

In [13]:
train_raw_nasa

,0,1,2,3,4,5,6,7,8,9,...,18412,18413,18414,18415,18416,18417,18418,18419,18420,LABEL
8826168flux,0.999893,0.999761,0.999637,0.999654,0.999918,0.999896,0.999825,0.999572,0.999741,0.999534,...,1.000255,1.000048,1.000276,1.000009,0.999960,1.000047,1.000487,1.000352,1.000200,2
4454752flux,0.988265,0.988611,0.988121,0.988514,0.988429,0.988452,0.988410,0.988485,0.988429,0.988762,...,0.995817,0.995702,0.995817,0.995938,0.995845,0.996309,0.996773,0.996398,0.996819,2
1575422flux,0.999639,0.999885,0.999638,0.999846,1.000053,0.999904,0.999786,0.999912,0.999914,0.999639,...,0.999973,0.999980,0.999929,0.999866,0.999863,1.000132,0.999983,1.000004,1.000043,1
1866174flux,1.000025,0.999938,1.000017,1.000123,0.999824,0.999785,0.999888,0.999910,1.000020,0.999789,...,0.999768,0.999970,1.000082,1.000071,1.000003,0.999845,0.999857,0.999907,1.000073,2
1720554flux,0.997872,0.997906,0.997869,0.998174,0.998398,0.998287,0.998114,0.998119,0.998134,0.998225,...,1.001853,1.002155,1.002716,1.003217,1.003229,1.002994,1.002570,1.002195,1.002029,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867844flux,1.000014,0.999871,0.999628,0.999610,0.999688,0.999612,0.999986,1.000058,1.000059,1.000069,...,1.000082,1.000049,0.999734,0.999954,0.999936,0.999964,1.000327,1.000056,1.000087,2
1577073flux,1.000092,1.000436,0.999789,1.000096,0.999888,0.998439,0.999836,1.000378,0.999535,0.999829,...,0.999822,0.999916,1.000314,1.000064,1.000444,1.000107,0.999220,1.000539,0.999780,1
1292087flux,1.000455,1.000369,1.000386,1.000033,1.000792,1.000430,1.000203,1.000362,1.000001,0.999923,...,1.000339,1.000234,1.000243,1.000138,1.000004,1.000145,0.999807,0.999710,0.999727,1
5807769flux,1.000339,1.000091,1.000140,1.000330,1.000002,1.000323,0.999690,1.000798,1.000043,1.000182,...,0.999990,0.999815,1.000036,0.999972,0.999600,0.999752,0.999797,0.999818,1.000123,2


In [11]:
train_raw_kaggle

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,1,-91.91,-92.97,-78.76,-97.33,-68.00,-68.24,-75.48,-49.25,-30.92,...,139.95,147.26,156.95,155.64,156.36,151.75,-24.45,-17.00,3.23,19.28
5083,1,989.75,891.01,908.53,851.83,755.11,615.78,595.77,458.87,492.84,...,-26.50,-4.84,-76.30,-37.84,-153.83,-136.16,38.03,100.28,-45.64,35.58
5084,1,273.39,278.00,261.73,236.99,280.73,264.90,252.92,254.88,237.60,...,-26.82,-53.89,-48.71,30.99,15.96,-3.47,65.73,88.42,79.07,79.43
5085,1,3.82,2.09,-3.29,-2.88,1.66,-0.75,3.85,-0.03,3.28,...,10.86,-3.23,-5.10,-4.61,-9.82,-1.50,-4.65,-14.55,-6.41,-2.55


In [ ]:
train_data, test_data = get_data(raw=1)

In [ ]:
train_data

In [15]:
pd.read_csv('../nasaTrain.csv', index_col='KepID')

,0,1,2,3,4,5,6,7,8,9,...,18412,18413,18414,18415,18416,18417,18418,18419,18420,LABEL
KepID,,,,,,,,,,,,,,,,,,,,,
8826168flux,0.999893,0.999761,0.999637,0.999654,0.999918,0.999896,0.999825,0.999572,0.999741,0.999534,...,1.000255,1.000048,1.000276,1.000009,0.999960,1.000047,1.000487,1.000352,1.000200,2
4454752flux,0.988265,0.988611,0.988121,0.988514,0.988429,0.988452,0.988410,0.988485,0.988429,0.988762,...,0.995817,0.995702,0.995817,0.995938,0.995845,0.996309,0.996773,0.996398,0.996819,2
1575422flux,0.999639,0.999885,0.999638,0.999846,1.000053,0.999904,0.999786,0.999912,0.999914,0.999639,...,0.999973,0.999980,0.999929,0.999866,0.999863,1.000132,0.999983,1.000004,1.000043,1
1866174flux,1.000025,0.999938,1.000017,1.000123,0.999824,0.999785,0.999888,0.999910,1.000020,0.999789,...,0.999768,0.999970,1.000082,1.000071,1.000003,0.999845,0.999857,0.999907,1.000073,2
1720554flux,0.997872,0.997906,0.997869,0.998174,0.998398,0.998287,0.998114,0.998119,0.998134,0.998225,...,1.001853,1.002155,1.002716,1.003217,1.003229,1.002994,1.002570,1.002195,1.002029,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867844flux,1.000014,0.999871,0.999628,0.999610,0.999688,0.999612,0.999986,1.000058,1.000059,1.000069,...,1.000082,1.000049,0.999734,0.999954,0.999936,0.999964,1.000327,1.000056,1.000087,2
1577073flux,1.000092,1.000436,0.999789,1.000096,0.999888,0.998439,0.999836,1.000378,0.999535,0.999829,...,0.999822,0.999916,1.000314,1.000064,1.000444,1.000107,0.999220,1.000539,0.999780,1
1292087flux,1.000455,1.000369,1.000386,1.000033,1.000792,1.000430,1.000203,1.000362,1.000001,0.999923,...,1.000339,1.000234,1.000243,1.000138,1.000004,1.000145,0.999807,0.999710,0.999727,1


In [ ]:
plt.plot(X_train.iloc[700])

In [ ]:
# exo_data.shape, non_exo_data.shape

In [ ]:
# all_data = pd.concat([exo_data, non_exo_data])

In [ ]:
temp2 = temp[temp>-660].count()

In [ ]:
min_length_frac = temp2[temp2==minim].count()/all_data.shape[0]

In [ ]:
majority_frac = temp2[temp2==18422].count()/all_data.shape[0]+temp2[temp2==18421].count()/all_data.shape[0]

In [ ]:
snip_len = 18422

In [ ]:
cols = [
    'Min snippet length',
    'Fraction of samples of min snippet length',
    'Fraction of samples of min snippet length+1||min snippet length',
    'Fraction of samples of greater length',
    'Chosen snippet length'
]
dats = [
    minim,
    min_length_frac,
    majority_frac,
    1-majority_frac,
    snip_len
]

In [ ]:
pd.DataFrame(dats, index=cols)

In [14]:
# minim = temp[temp>-660].count().min()
train_raw_nasa.to_csv('../nasaTrain.csv', index=True, index_label='KepID')
test_raw_nasa.to_csv('../nasaTest.csv', index=True, index_label='KepID')

In [17]:
!pip install pyts

  Using cached pyts-0.12.0-py3-none-any.whl (2.5 MB)
  Using cached numba-0.54.1-cp38-cp38-macosx_10_14_x86_64.whl (2.3 MB)
  Using cached llvmlite-0.37.0-cp38-cp38-macosx_10_9_x86_64.whl (19.1 MB)


In [21]:
DEFAULT_LEN = 18421

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ExoHunter.cleaner import Cleaner
from ExoHunter.formatter import Formatter
from ExoHunter.params import *

cleaner = Cleaner()
formatter = Formatter()

train_data, test_data = cleaner.get_data(drive=0)

X, y = cleaner.get_Xy(train_data)

In [89]:
def random_selector(data):
    desired_len = np.random.randint(2000, 6000)
    X = data.drop(columns='LABEL')
    y = data['LABEL']
    rand_upper = DEFAULT_LEN-desired_len
    lower = np.random.randint(0, rand_upper)
    X_ = X.sample(frac=1)
    X_ = X_.iloc[:,lower:lower+desired_len]
    data_ = X_.join(y)
    return data_

In [92]:
X_selected = random_selector(train_data)
X_selected

,5638,5639,5640,5641,5642,5643,5644,5645,5646,5647,...,9016,9017,9018,9019,9020,9021,9022,9023,9024,LABEL
KepID,,,,,,,,,,,,,,,,,,,,,
1025494flux,0.999880,0.999719,0.999858,0.999844,0.999875,1.000016,0.999998,1.000009,0.999871,0.999923,...,1.000003,1.000112,1.000096,1.000104,1.000085,1.000161,1.000060,1.000152,1.000073,1
1025578flux,0.999703,0.999785,0.999799,0.999900,0.999742,0.999951,0.999875,1.000016,0.999505,0.999683,...,0.999480,0.999713,0.999980,1.000033,0.999697,0.999878,0.999821,1.000175,0.999838,1
1025986flux,0.998692,0.998506,0.998445,0.998360,0.998309,0.998108,0.997917,0.997754,0.997594,0.997456,...,0.998392,0.998171,0.998217,0.998269,0.998462,0.998387,0.998361,0.998571,0.998407,1
1026032flux,0.997490,1.001007,0.998850,0.998357,0.998363,0.998266,0.998267,0.998734,0.998087,0.998749,...,1.001162,1.000072,1.001563,1.000730,1.001486,1.001996,1.001324,1.001480,1.001009,1
1026133flux,1.000001,0.999934,1.000194,1.000126,0.999707,0.999855,1.000015,0.999829,0.999690,0.999775,...,1.000136,1.000062,1.000252,1.000095,1.000166,1.000046,1.000019,1.000013,1.000146,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9146018flux,0.999473,1.000105,0.999726,0.999868,0.999692,0.999853,0.999685,0.999811,0.999531,0.999739,...,1.000382,1.000232,1.000343,1.000349,1.000196,1.000250,1.000180,1.000288,0.999797,2
9153570flux,0.998244,0.998538,0.998678,0.998935,0.998451,0.998346,0.997846,0.998281,0.998249,0.998094,...,0.993048,0.993403,0.992014,0.991395,0.993185,0.993180,0.993298,0.993656,0.993888,2
9157634flux,1.000074,1.000172,0.999800,0.999860,1.000130,0.999946,0.999504,0.999056,0.998983,0.999363,...,0.999478,0.999567,0.999342,0.999709,0.999725,0.999724,0.999332,0.999395,0.998956,2


In [93]:
def selection_to_csv(data, n):
    X_selected = random_selector(data)
    X_selected.to_csv(f'../processed_data/nasa/nasaSelect{n}.csv', index=True, index_label='KepID')

In [96]:
for n in range(10):
    print(f'Writing CSV no.{n}')
    selection_to_csv(train_data, n)

Writing CSV no.0
Writing CSV no.1
Writing CSV no.2
Writing CSV no.3
Writing CSV no.4
Writing CSV no.5
Writing CSV no.6
Writing CSV no.7
Writing CSV no.8
Writing CSV no.9


In [95]:
X_selected

,5638,5639,5640,5641,5642,5643,5644,5645,5646,5647,...,9016,9017,9018,9019,9020,9021,9022,9023,9024,LABEL
KepID,,,,,,,,,,,,,,,,,,,,,
1025494flux,0.999880,0.999719,0.999858,0.999844,0.999875,1.000016,0.999998,1.000009,0.999871,0.999923,...,1.000003,1.000112,1.000096,1.000104,1.000085,1.000161,1.000060,1.000152,1.000073,1
1025578flux,0.999703,0.999785,0.999799,0.999900,0.999742,0.999951,0.999875,1.000016,0.999505,0.999683,...,0.999480,0.999713,0.999980,1.000033,0.999697,0.999878,0.999821,1.000175,0.999838,1
1025986flux,0.998692,0.998506,0.998445,0.998360,0.998309,0.998108,0.997917,0.997754,0.997594,0.997456,...,0.998392,0.998171,0.998217,0.998269,0.998462,0.998387,0.998361,0.998571,0.998407,1
1026032flux,0.997490,1.001007,0.998850,0.998357,0.998363,0.998266,0.998267,0.998734,0.998087,0.998749,...,1.001162,1.000072,1.001563,1.000730,1.001486,1.001996,1.001324,1.001480,1.001009,1
1026133flux,1.000001,0.999934,1.000194,1.000126,0.999707,0.999855,1.000015,0.999829,0.999690,0.999775,...,1.000136,1.000062,1.000252,1.000095,1.000166,1.000046,1.000019,1.000013,1.000146,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9146018flux,0.999473,1.000105,0.999726,0.999868,0.999692,0.999853,0.999685,0.999811,0.999531,0.999739,...,1.000382,1.000232,1.000343,1.000349,1.000196,1.000250,1.000180,1.000288,0.999797,2
9153570flux,0.998244,0.998538,0.998678,0.998935,0.998451,0.998346,0.997846,0.998281,0.998249,0.998094,...,0.993048,0.993403,0.992014,0.991395,0.993185,0.993180,0.993298,0.993656,0.993888,2
9157634flux,1.000074,1.000172,0.999800,0.999860,1.000130,0.999946,0.999504,0.999056,0.998983,0.999363,...,0.999478,0.999567,0.999342,0.999709,0.999725,0.999724,0.999332,0.999395,0.998956,2
